In [ ]:
import os
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
# We opt to define all the global configuration settings through this dict
config = {"resampling_fs" : 22050, #expected resampling rate on HDF5 file creation
          "dst_path":"./choral_dst", # User Dataset path (if None, the default one will be used)
          "hdf5_filepath":"./choral_dst.hdf5",
          "batch_size_train": 512,
          "batch_size_test": 256,
          "num_frames": 132300 # 8 sec @ 22050Hz
         }

if not os.path.isdir('./debug'):
    os.mkdir('./debug')

<h1><center>Data Handling Methodology</center></h1>

For the first step of this task, we will work from the __Choral Singing Dataset__, which contains three choir songs composed of 4 groups of 4 singers (16 parts per song), for a total of 48 stems. The dataset can be downloaded from Zenodo at the following:

https://zenodo.org/record/2649950#.XlJRNy2ZPRY

If you happen to already have the dataset locally, please define it's local path through the ***dst_path*** variable. Otherwise, the cell below will take care of downloading it for you:
***
### 1. Dataset Download:

In [ ]:
#This cell downloads the Mridangam Stroke Dataset in its entirety
import urllib.request
import zipfile
import os, sys,shutil

#Link for the dataset. If you already have the dataset locally, define its location through the config dict above 
url = 'https://zenodo.org/record/2649950/files/ChoralSingingDataset.zip'

if config["dst_path"] == None:
    main_data_dir = './choral_dst'
    if not os.path.exists(main_data_dir): #creating the directory if not exist
        os.mkdir(main_data_dir)
    print('Downloading Choral Singing Dataset Dataset...')
    foldername = url.split('/')[-1]
    urllib.request.urlretrieve(url,foldername)
    #Unzipping to a specific folder
    zip_ref = zipfile.ZipFile(foldername, 'r')
    zip_ref.extractall(main_data_dir)
    zip_ref.close()
    os.remove(foldername)#Removing the zip file
    print('Data downloaded and unzipped to: ',main_data_dir)
else:
    print('User Path Defined. Dataset Expected to be Found at the Following:\n'+config["dst_path"])
 

***
### 2. Storing our Data in a Convenient Format

For the sake of simplicity and clarity, we will store our song stems in a **HDF5 file**, which will help us to manage the later steps a bit more easily. This hierarchical file format will allow us to store our data in an intuitive way as follow:

**[HDF5_File] -> [Set] -> [Song] -> [Part]**

Since our dataset is of *very* limited size, we will break it down as follow:

| Set   | Stems | Songs |
| :---:   | :---: | :---: |
| Train |    24   |    A, B   |
| Valid |   8    |   A, B    |
| Test  |16| C |

**Note:** By default, the Choral Singing Dataset follow the following naming convention:

*DatasetName_SongAcronym_PartName_PartNumber.wav*

In [ ]:
import h5py
import librosa
import soundfile
import glob
import random
from tqdm import tqdm
from time import sleep
from pathlib import Path

# First check if HDF5 already exists:
hdf5_file = Path(config['hdf5_filepath'])
if not hdf5_file.is_file():
    tracks = dict()
    resampling_fs = config['resampling_fs']
    dst_path = './choral_dst' if (config['dst_path'] == None) else config['dst_path']

    # Get all the stem files
    dst_stems = glob.glob(dst_path+"/*.wav",recursive=False)
    # Extract only songs from name
    songs = set([os.path.splitext(os.path.basename(i))[0].split('_')[1] for i in dst_stems])
    # Take 2/3 of songs for train/valid
    train_val_songs = random.sample(songs, int(np.ceil(0.666*len(songs))))
    # Take the rest as test set
    test_song = list(set(songs) - set(train_val_songs))
    test_idx = [int(i) for i, track in enumerate(dst_stems) if [True for song in test_song if str('_'+song+'_') in track]]
    # Take one random group of singer per song for each song as validation set
    rand_part = random.randint(1,4)
    val_idx   = [int(i) for i, track in enumerate(dst_stems) if [True for song in train_val_songs if ((str('_'+song+'_') in track) and (str('_'+str(rand_part)+'.wav')) in track)]]
    # Deduce the train set from valid / test set
    train_idx = list(set(range(len(dst_stems))) -  (set(val_idx + test_idx)))

    # tracks['train'] = np.take(dst_stems, train_idx)
    tracks['train'] = np.take(dst_stems, train_idx)
    tracks['valid'] = np.take(dst_stems, val_idx)
    tracks['test'] = np.take(dst_stems, test_idx)

    print('Ready to create HDF5')

    h5file = h5py.File(config["hdf5_filepath"], "w")

    # Write stems to hdf5 file for train/valid/test partitions
    for curr_partition in ["train", "valid", "test"]:

        print("Writing " + curr_partition + " partition with "+str(len(tracks[curr_partition]))+" files...")

        stem_list = tracks[curr_partition]

        # Create group for set if needed
        if not str(curr_partition) in h5file:
            set_grp  = h5file.create_group(curr_partition)

        sleep(1)

        for track in tqdm(stem_list):

            filename = os.path.splitext(os.path.basename(track))[0].split('_')

            song = filename[1]
            part = ''.join(filename[2:4])

            # Create group for the Song if needed
            if not str(curr_partition+'/'+song) in h5file:
                song_grp  = set_grp.create_group(song)

            # Create group for the Part if needed
            if not str(curr_partition+'/'+song+'/'+part) in h5file:
                part_grp = h5file[str(curr_partition+'/'+song)]
                subgrp  = part_grp.create_group(part)

            # Once part groups / song subgroups are created, store file
            audio, s = librosa.load(track, sr=resampling_fs)
            subgrp = h5file[str(curr_partition+'/'+song+'/'+part)]
            subgrp.create_dataset("raw_wav",data=audio)

    print('Done Creating HDF5 at path: '+config['hdf5_filepath'])
    h5file.close()
else:
    print('HDF5 Already Exists. File Expected to be Found at the Following:\n'+config["hdf5_filepath"])

***
### 3. Defining the Generator Function:

Instead of computing features on the entirety of each of these songs, we will create many song snippets (or frames), which will depict different possible musical settings (SAT, ATB, SB, etc.) out of the songs. This will allow us to cover all possible singer permutations on a per-frame basis, without having to compute and feed to the model substantial chunk of audio.

The next step would be to define the function that will generate these song frames for us. 

**Note:** This function will yield the song segments but won't render them to disk. We can proceed by computing everything on the fly and save ourselves some local memory space.

In [ ]:
def batchGenerator(hdf5_filepath, batch_size, num_frames, partition='train', debug=False):
    
    dataset   = h5py.File(hdf5_filepath, "r")
    itCounter = 0

    itCounter = itCounter + 1
    
    # Choose random song and retrieve all available part for it
    randsong = random.choice(list(dataset[partition].keys())) # Get random song from dataset keys
    sources  = list(dataset[partition][randsong].keys())      # Get all available sources for song
    
    # Compute available parts per group
    part_per_group = dict({'soprano':None,'alto':None,'tenor':None,'bass':None})
    for group in part_per_group.keys():
        parts = [source[-1] for source in sources if group in source]
        part_per_group[group] = parts

    # Init the start/end values of song snippet
    startspl = 0
    endspl   = 0

    # Preping the output shape of the generator
    out_shapes = {'mix':np.zeros((batch_size, num_frames)),'num_sing':np.zeros((batch_size),dtype=int)}

    # Will create batch_size x mixes of unique singers permutations
    for i in range(batch_size):

        randsources = random.sample(part_per_group.keys(), random.randint(1,4)) # Randomize source pick

        # Get Start and End samples. Pick random part to calculate start/end spl
        randpart = random.choice(sources)
        startspl = random.randint(0,len(dataset[partition][randsong][randpart]['raw_wav'])-num_frames) # This assume that all stems are the same length
        endspl   = startspl+num_frames

        # Get Random Sources with part number: 
        randsources_with_part = ["{}{}".format(a, random.choice(part_per_group[a])) for a in randsources] # Concatenate strings for part name
        # Retrieve the chunks and store them in output shapes                                         
        for source in randsources_with_part:

            source_chunk = dataset[partition][randsong][source]['raw_wav'][startspl:endspl]              # Retrieve part's chunk
            out_shapes['mix'][i] = np.add(out_shapes['mix'][i],source_chunk)            # Add the chunk to the mix

        # Scale down the mix based off number of sources
        scaler = len(randsources_with_part)
        out_shapes['mix'][i] = (out_shapes['mix'][i]/scaler)
        out_shapes['num_sing'][i] = len(randsources_with_part)

    # if debug == True, pick one random mix from batch and synthesize it
    if debug==True:

        rand_pick = random.randint(0,batch_size-1)
        debug_dir = './debug/it#'+str(itCounter)+'_batchpick#'+str(rand_pick)
        if not os.path.isdir(debug_dir):
            os.mkdir(debug_dir)
        soundfile.write(debug_dir+'/'+'mix_'+str(out_shapes['num_sing'][rand_pick])+'_singers_'+'.wav', out_shapes['mix'][rand_pick], config['resampling_fs'], 'PCM_24')
            
    
    return (out_shapes['mix'], out_shapes['num_sing'])

As you can see, the generator function returns a dictionary of shape:<br>
```out_shapes = {'mix':np.zeros((batch_size, num_frames)),'num_sing':np.zeros((batch_size))}``` <br><br>where:
* the key ```'mix'``` holds ```<batch_size>``` number of snippet of size ```<num_frames>```
* the key ```'num_sing'``` holds the ground truth respective to each of these mixes

<br>The purpose of the generator function is to strictly provide the time-domain snippets. For the pre-processing steps related to frequency-domain conversion and more, see the next section below:

***
<h1><center>Features Extraction Step</center></h1>

The first step of our experiment entails training a deep neural network model using some low-level spectral features extracted using the audio analysis library **Essentia**. 

***
### 1. Computing Time-Domain Snippets

In [ ]:
num_frames = config['num_frames'] # size of song snippets

audio_data_train, y_train = batchGenerator(config['hdf5_filepath'], config['batch_size_train'], num_frames, partition='train', debug=True)
print('Audio Data for Training Loaded with Shape:\n'+ str(np.shape(audio_data_train)))
print('Ground Truth for Training Loaded with Shape:\n'+ str(np.shape(y_train)))
audio_data_test, y_test = batchGenerator(config['hdf5_filepath'], config['batch_size_test'], num_frames, partition='test', debug=True)
print('Audio Data for Testing Loaded with Shape:\n'+ str(np.shape(audio_data_test)))
print('Ground Truth for Testing Loaded with Shape:\n'+ str(np.shape(y_test)))

Now our audio data are loaded into a 2D numpy array called ```audio_data``` and our ground truth is loaded onto a 1D numpy array called ```y```. <br>

For what it is worth, let's plot a few audio snippets labeled with their respective ground truth:

In [ ]:
plt.figure(1, figsize=(10,100))
plt.subplots(4,1,figsize=(20,30))
plt.subplots_adjust(hspace=0.5)

for i in range(4):
    
    rand_snippet = random.randint(0,np.shape(audio_data_train)[0]-1) # Pick random snippet from audio_data
    x = audio_data_train[rand_snippet]
    num_singer = y_train[rand_snippet]

    plt.subplot(4, 1, (i+1))
    plt.plot(x)
    plt.title(r"$\bf{""Singers: "+str(num_singer)+ "}$")

***
### 2. Feature Extraction

Now is the time to actually extract the features from all the audio snippets we've just computed. We will exploit the fact that each singer in a SATB recording settings each have their respective singing range (that is to say, there is still a fair amount of harmonic overlap). <br>A first naive approach, would be to compute the __MFCC coefficients__ in the hope that this feature would accurately depict the recordings' frequency range in respect to the number of singers singing. Moreover, this feature seems like a good fit for our task as the signals we are dealing with only entail singing voice.

In [ ]:
from essentia.standard import *

x = essentia.array(audio_data_train[0])
MonoWriter(filename ='./webgwegweg.wav',sampleRate=22050)(x)
features, features_frames = MusicExtractor(lowlevelSilentFrames='drop',
                                                  lowlevelFrameSize = 2048,
                                                  lowlevelHopSize = 1024,
                                                  lowlevelStats = ['mean', 'stdev'])('./webgwegweg.wav')
scalar_lowlevel_descriptors = [descriptor for descriptor in features.descriptorNames() if 'lowlevel' in descriptor and isinstance(features[descriptor], float)]
print("Subset of features to be considered:\n",scalar_lowlevel_descriptors)

In [ ]:
def createDataForPartition(audio,y,partition='train'):
    
    w = Windowing(type = 'hann')
    spectrum = Spectrum()
    mfcc = MFCC(numberCoefficients=13)
    logNorm = UnaryOperator(type='log')

    mfccs = np.empty([np.shape(audio)[0],int((config['num_frames']/512)),13])
    melbands_log = np.empty([np.shape(audio)[0],int((config['num_frames']/512)),40])

    sleep(1.0)

    # Compute MFCCS for all snippets: shape <snippet, coefficients>
    total_features = []
    data_file = os.path.join('./',partition+'_data.csv')
    with open(data_file, 'w') as writer:
        
        #adding column names as the first line in csv
        line2write = ','.join(scalar_lowlevel_descriptors + ['num_sing']).replace('lowlevel.','') + '\n'
        writer.write(line2write)
        
        for idx in tqdm(range(np.shape(audio)[0])):

            x = essentia.array(audio[idx])
            MonoWriter(filename ='./webgwegweg.wav',sampleRate=22050)(x)

            try:
                features, features_frames = MusicExtractor(lowlevelSilentFrames='drop',
                                                          lowlevelFrameSize = 2048,
                                                          lowlevelHopSize = 1024,
                                                          lowlevelStats = ['mean', 'stdev'])('./webgwegweg.wav')
                selected_features = [features[descriptor] for descriptor in scalar_lowlevel_descriptors]
                total_features.append(selected_features)
                line2write = str(selected_features)[1:-1] + ',' + str(y[idx]) + '\n'
                writer.write(line2write)
            except:
                audio[idx] = 0
                continue
            os.remove('./webgwegweg.wav')

Once we compute MFCCs for all audio snippet, let's plot some of them to make sure they look as expected:

In [ ]:
print('Computing Low Level Features for training set:')
createDataForPartition(audio_data_train,y_train,partition='train')
print('Computing Low Level Features for test set:')
createDataForPartition(audio_data_test,y_test,partition='test')

In [ ]:
import seaborn as sns
import pandas as pd
from sklearn import preprocessing
from sklearn.preprocessing import OneHotEncoder

def retrieveDataFromCSV(data_path):
    #Read data
    data = pd.read_csv(data_path)
    #Let's see the first lines of our data
    data.head()
    #sns.pairplot(data.iloc[:, -7:], hue = "num_sing");

    data_modif = data.copy()

    #min_max_scaler = preprocessing.MinMaxScaler()
    #data_modif.iloc[:,:84] = min_max_scaler.fit_transform(data.iloc[:,:84].values)
    data_modif.describe()
    data_modif.num_sing.value_counts()

    #input values put in a matrix, there are 84 features
    X = data_modif.iloc[:,:84].values 
    #Creating output values
    y = np.array(data_modif.num_sing) #create label encoded outputs
    
    onehot_encoder = OneHotEncoder(sparse=False)
    y_onehot = onehot_encoder.fit_transform(y.reshape(len(y), 1))
    
    return X, y_onehot

In [ ]:
# Couple of modules for wrapped feature selection
from sklearn.ensemble import RandomForestClassifier # wrapper-based
from sklearn.feature_selection import SelectFromModel

X_train, y_train = retrieveDataFromCSV('train_data.csv')
X_test, y_test   = retrieveDataFromCSV('test_data.csv')

# Finally, we end up with these shapes:
print('Pre Selection Audio Data for Training with Shape:\n'+ str(np.shape(X_train)))
print('Pre Selection Ground Truth for Training Loaded with Shape:\n'+ str(np.shape(y_train)))
print('Pre Selection Audio Data for Testing with Shape:\n'+ str(np.shape(X_test)))
print('Pre Selection Ground Truth for Testing Loaded with Shape:\n'+ str(np.shape(y_test)))

In [ ]:
from keras.models import Sequential
from keras import layers

def compose_model(num_features):
    
    model = Sequential()

    # Input layer 
    model.add(layers.BatchNormalization(name='InputLayer', input_shape=(num_features,)))
    
    # 1. hidden layer
    model.add(layers.Dense(name='HiddenLayer_1', units = 40))
    model.add(layers.BatchNormalization())
    model.add(layers.Activation('tanh'))
    model.add(layers.Dropout(0.3))

    # 2. hidden layyer
    model.add(layers.Dense(name='HiddenLayer_2', units = 20))
    model.add(layers.BatchNormalization())
    model.add(layers.Activation('relu'))
    model.add(layers.Dropout(0.3))

    # Output layer
    model.add(layers.Dense(name='Output_layer', units = 4))
    model.add(layers.Activation('sigmoid'))
    
    return model

model = compose_model(X_train.shape[1])

In [ ]:
from sklearn.metrics import classification_report

model.summary()
model.compile(optimizer = 'Adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
summary = model.fit(X_train, y_train, batch_size = 50, epochs = 250, validation_split=0.2, verbose=0,)

score = model.evaluate(X_test, y_test, verbose = 0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])
print(score)

***
<h1><center>Deep Salience Approach</center></h1>

Now that we have computed our baseline, which as you could see left room for great improvement, we can proceed in digging into the core of this experiment: __using a deep salience approach to estimate the number of singing sources__. 

Our experiment will heavily rely on the experiment conducted by Bittner, R et. al tackling the issue of multi-F0 estimation in polyphonic music, using Deep Neural Networks (see: *Deep Salience Representations for F0 Estimation in Polyphonic Music - https://bmcfee.github.io/papers/ismir2017_salience.pdf*).

In fact, we will use the companion code presented in the above-mentioned paper in order to estimate the number of simultaneous singing sources: https://github.com/rabitt/ismir2017-deepsalience

***
### 1. Importing Required Python File - Defining Function Parameters

In [1]:
# This file works as a standalone processor (no need for other file dependencies)
import predict_on_audio as ds
from collections import Counter
import tensorflow as tf

Using TensorFlow backend.


In [2]:
# Arg Dict passed to the DeepSalience main function
model_path = './weights/model.ckpt'
args = {'audio':None, # audio chunk np array
        'task':'multif0', # "all, bass, melody1, melody2, melody3, multif0, pitch, vocal."
        'save_dir':'./', # no used
        'use_neg':True, 
        'threshold':0.7, # IMPORTANT
        'output_format':'multif0', # ['singlef0', 'multif0', 'salience']
        'model':None} # Hold the deep salience model

In [3]:
import librosa
# np.set_printoptions(threshold=sys.maxsize)

# audio_data_test_new = audio_data_test[~np.all(audio_data_test == 0, axis=1)]
# y_predict = np.zeros(np.shape(audio_data_test_new)[0])

# Preload model 
init_op = tf.group(tf.global_variables_initializer(), tf.local_variables_initializer())
sess=tf.Session()
sess.run(init_op)
#First let's load meta graph and restore weights
saver = tf.train.import_meta_graph('./weights/model.ckpt-4590.meta')
saver.restore(sess,'./weights/model.ckpt-4590')

def DeepSalience(input_, is_train):
    conv1 = tf.layers.conv2d(input_, 128, (5, 5), strides=(1, 1), padding='same', name="conv_1", activation = tf.nn.relu)

    conv1 = tf.layers.batch_normalization(conv1, training=is_train)

    conv2 = tf.layers.conv2d(conv1, 64, (5, 5), strides=(1, 1), padding='same', name="conv_2", activation = tf.nn.relu)

    conv2 = tf.layers.batch_normalization(conv2, training=is_train)

    conv3 = tf.layers.conv2d(conv2, 64, (3, 3), strides=(1, 1), padding='same', name="conv_3", activation = tf.nn.relu)

    conv3 = tf.layers.batch_normalization(conv3, training=is_train)

    conv4 = tf.layers.conv2d(conv3, 64, (3, 3), strides=(1, 1), padding='same', name="conv_4", activation = tf.nn.relu)

    conv4 = tf.layers.batch_normalization(conv4, training=is_train)

    conv5 = tf.layers.conv2d(conv4, 64, (3, 70), strides=(1, 1), padding='same', name="conv_5", activation = tf.nn.relu)

    conv5 = tf.layers.batch_normalization(conv5, training=is_train)

    final_layer = tf.layers.conv2d(conv5, 1, (1, 1), strides=(1, 1), padding='same', name="conv_6", activation = None)

    return tf.squeeze(final_layer)

batch_size = 1
max_phr_len = 517
cqt_bins = 360
is_train = False
input_placeholder = tf.placeholder(tf.float32, shape=(batch_size, max_phr_len, cqt_bins, 6), name='input_placeholder')
output_logits = DeepSalience(input_placeholder, is_train)
outputs = tf.nn.sigmoid(output_logits)

# for idx in tqdm(range(np.shape(audio_data_test_new)[0])):
    
#     args['audio'] = audio_data_test[idx]
#     times, freqs  = ds.main(args)
#     # Counting the simultaneous number of singers from the multi_f0 output
#     multi_f0 = np.asarray([len(freqs[idx]) for idx in range(len(freqs)) if (len(freqs[idx])>0)])
#     if len(multi_f0)==0:
#         multi_f0=[0]
#     # Probably better ways to do this, but for now let's stick to it

#     #y_predict[idx] = int(np.around(np.mean(multi_f0)))
#     y_predict[idx] = Counter(multi_f0).most_common(1)[0][0]

# # Make sure that num_singer is within range
# y_predict[y_predict < 1] = 1
# y_predict[y_predict > 4] = 4
# onehot_encoder = OneHotEncoder(sparse=False)
# y_predict_onehot = onehot_encoder.fit_transform(y_predict.reshape(len(y_predict), 1))

INFO:tensorflow:Restoring parameters from ./weights/model.ckpt-4590
Instructions for updating:
Use `tf.keras.layers.Conv2D` instead.
Instructions for updating:
Please use `layer.__call__` method instead.
Instructions for updating:
Use keras.layers.BatchNormalization instead.  In particular, `tf.control_dependencies(tf.GraphKeys.UPDATE_OPS)` should not be used (consult the `tf.keras.layers.batch_normalization` documentation).


In [4]:
import numpy as np
def extract_f0_file(file_name, sess):
    
    y,sr = librosa.load('./mix_3_singers_.wav', sr=22050)
    print(len(y))
    in_batches_hcqt, _, nchunks_in = ds.compute_hcqt(y)
    print(in_batches_hcqt.shape)
    in_batches_hcqt = in_batches_hcqt.reshape(batch_size, max_phr_len, cqt_bins,in_batches_hcqt.shape[0])
    #in_batches_hcqt = np.swapaxes(in_batches_hcqt, -1, -2)
    out_batches_atb = []
    #for in_batch_hcqt in in_batches_hcqt:
    feed_dict = {input_placeholder: in_batches_hcqt}
    out_atb = sess.run(outputs, feed_dict=feed_dict)
    out_batches_atb.append(out_atb)
    out_batches_atb = np.array(out_batches_atb)

    out_batches_atb = out_batches_atb[:atb.shape[0]]
    #     time_2, est_freq = utils.process_output(out_batches_atb)

extract_f0_file('./',sess)

132300
(6, 360, 517)


FailedPreconditionError: Attempting to use uninitialized value conv_1/kernel
	 [[node conv_1/kernel/read (defined at /usr/local/lib/python3.7/site-packages/tensorflow_core/python/framework/ops.py:1748) ]]

Original stack trace for 'conv_1/kernel/read':
  File "/usr/local/Cellar/python/3.7.5/Frameworks/Python.framework/Versions/3.7/lib/python3.7/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/local/Cellar/python/3.7.5/Frameworks/Python.framework/Versions/3.7/lib/python3.7/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/usr/local/lib/python3.7/site-packages/traitlets/config/application.py", line 664, in launch_instance
    app.start()
  File "/usr/local/lib/python3.7/site-packages/ipykernel/kernelapp.py", line 563, in start
    self.io_loop.start()
  File "/usr/local/lib/python3.7/site-packages/tornado/platform/

In [ ]:
print(np.shape(audio_data_test)[0])
print(y_test)
print(y_predict_onehot)
from sklearn.metrics import classification_report
print(classification_report(y_test, y_predict_onehot))